## Setup

In [33]:
import datetime
import logging
import os
import subprocess
import time
import google.cloud.storage as gcs

from googleapiclient import discovery
from googleapiclient import errors
from googleapiclient import http

BUCKET="dataflow-jlewi-cloud-ml"
PROJECT="dataflow-jlewi"
logging.getLogger().setLevel(logging.INFO)

cloudml = discovery.build(
      'ml',
      'v1',
      requestBuilder=http.HttpRequest)


## Create Training Data

In [ ]:
# Make sure you're logged in and have credentials
command = ["python", "-m", "running_average.create_training_data",
           "--output_path=gs://{0}/running_average/inputs.tfrecord".format(BUCKET)]

output = subprocess.check_output(command,stderr=subprocess.STDOUT)
print output

## Submit Training Job

In [21]:
from running_average import submit_training_job
reload(submit_training_job)
now = datetime.datetime.now()
datestamp = now.strftime("%Y%m%d_%H%M%S")
job_name="running_average_{0}".format(datestamp)
output_path = "gs://{0}/running_average/{1}/output/results".format(BUCKET, datestamp)
main_args = ["--train_data_path=gs://{0}/running_average/inputs.tfrecord".format(BUCKET)]
final_job = submit_training_job.submit_job(job_name, main_args, output_path, PROJECT)

INFO:root:Setup file: /content/datalab/home/git_cmle_simple/setup.py
INFO:root:Found packages: ['/tmp/tmp0zExRH/running_average-0.1.1.tar.gz']
INFO:oauth2client.client:Attempting refresh to obtain initial access_token
INFO:oauth2client.client:Refreshing access_token
INFO:root:Copying /tmp/tmp0zExRH/running_average-0.1.1.tar.gz to gs://dataflow-jlewi-cloud-ml/running_average/20170826_182755/output/results/staging/running_average-0.1.1.tar.gz
INFO:googleapiclient.discovery:URL being requested: POST https://ml.googleapis.com/v1/projects/dataflow-jlewi/jobs?alt=json
INFO:oauth2client.client:Attempting refresh to obtain initial access_token
INFO:oauth2client.client:Refreshing access_token
INFO:root:Set response:
{
  "createTime": "2017-08-26T18:27:57Z",
  "jobId": "running_average_20170826_182755",
  "state": "QUEUED",
  "trainingInput": {
    "args": [
      "--train_data_path=gs://dataflow-jlewi-cloud-ml/running_average/inputs.tfrecord",
      "--output_path=gs://dataflow-jlewi-cloud-ml/r

## Save Model For Serving

In [24]:
from running_average import build_saved_model
reload(build_saved_model)
saved_model = "gs://{0}/running_average/{1}/saved_model".format(BUCKET, datestamp)
build_saved_model.save_model(output_path, saved_model)

INFO:root:Exporting the model to gs://dataflow-jlewi-cloud-ml/running_average/20170826_182755/saved_model
INFO:root:Restoring weights from: gs://dataflow-jlewi-cloud-ml/running_average/20170826_182755/output/results/save_dir-5


INFO:tensorflow:Restoring parameters from gs://dataflow-jlewi-cloud-ml/running_average/20170826_182755/output/results/save_dir-5


INFO:tensorflow:Restoring parameters from gs://dataflow-jlewi-cloud-ml/running_average/20170826_182755/output/results/save_dir-5
INFO:root:inputs: ParseExample/ParseExample:0


INFO:tensorflow:No assets to save.


INFO:tensorflow:No assets to save.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: gs://dataflow-jlewi-cloud-ml/running_average/20170826_182755/saved_model/saved_model.pb


INFO:tensorflow:SavedModel written to: gs://dataflow-jlewi-cloud-ml/running_average/20170826_182755/saved_model/saved_model.pb


## Deploy the model

In [32]:
endpoint = None
model_name = "simple_cmle"
model_path = "gs://dataflow-jlewi-cloud-ml/running_average/20170826_182755/saved_model"

now = datetime.datetime.now()
datestamp = now.strftime("%Y%m%d_%H%M%S")
model_version = 'v' + datestamp

deploy_model.deploy(PROJECT, model_path, model_name, model_version)


INFO:googleapiclient.discovery:URL being requested: POST https://ml.googleapis.com/v1/projects/dataflow-jlewi/models?alt=json
INFO:root:Model simple_cmle already exists.
INFO:googleapiclient.discovery:URL being requested: POST https://ml.googleapis.com/v1/projects/dataflow-jlewi/models/simple_cmle/versions?alt=json
INFO:root:Set response:
{
  "metadata": {
    "@type": "type.googleapis.com/google.cloud.ml.v1.OperationMetadata",
    "createTime": "2017-08-26T23:23:26Z",
    "modelName": "projects/dataflow-jlewi/models/simple_cmle",
    "operationType": "CREATE_VERSION",
    "version": {
      "createTime": "2017-08-26T23:23:26Z",
      "deploymentUri": "gs://dataflow-jlewi-cloud-ml/running_average/20170826_182755/saved_model",
      "etag": "RodNWVB0IDY=",
      "name": "projects/dataflow-jlewi/models/simple_cmle/versions/v20170826_232324",
      "state": "CREATING"
    }
  },
  "name": "projects/dataflow-jlewi/operations/create_simple_cmle_v20170826_232324-1503789806819"
}


## Send some predictions

In [35]:
from run
projects = cloudml.projects()

body = {
  'instances' : [
    {
      'inputs': [10],
    }
  ],
}
request = projects.predict(name='projects/{0}/models/{1}/versions/{2}'.format(PROJECT, model_name, model_version),
                           body=body)

response = request.execute()
logging.info('Set response:\n%s', util.PrettyFormat(response))

Help on method method in module googleapiclient.discovery:

method(self, **kwargs) method of googleapiclient.discovery.Resource instance
    Performs prediction on the data in the request.
    
    **** REMOVE FROM GENERATED DOCUMENTATION
    
    Args:
      name: string, Required. The resource name of a model or a version.
    
    Authorization: requires the `predict` permission on the specified resource. (required)
      body: object, The request body. (required)
        The object takes the form of:
    
    { # Request for predictions to be issued against a trained model.
          # 
          # The body of the request is a single JSON object with a single top-level
          # field:
          # 
          # <dl>
          #   <dt>instances</dt>
          #   <dd>A JSON array containing values representing the instances to use for
          #       prediction.</dd>
          # </dl>
          # 
          # The structure of each element of the instances list is determined by yo